<a href="https://colab.research.google.com/github/Labs123355/hey_world/blob/main/Carbon_Sequestration_Project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title
df_map

,Facility_Names,Facility_Latitude,Facility_Longitiude,Nearest Well Names,Well_Latitude,Well_Longitiude,Shortest Distance(km),Facility Coordinates,Well Coordinates
0,Alpaca Energy LLC,41.66,-76.81,DELCIOTTO,41.576682,-76.884716,11.153909,POINT (41.66 -76.81),POINT (41.576682 -76.884716)
1,"Armstrong Power, LLC",40.64,-79.35,T C CROWNOVER,40.611703,-79.437122,8.013133,POINT (40.64 -79.35),POINT (40.611703 -79.437122)
2,BELLEFIELD BOILER PLANT,40.44,-79.95,FAWN DEVELOPERS INC,40.648018,-79.777763,27.321533,POINT (40.44 -79.95),POINT (40.648018 -79.777763)
3,Beaver Dam Energy LLC,41.65,-76.80,DELCIOTTO,41.576682,-76.884716,10.778206,POINT (41.65 -76.8),POINT (41.576682 -76.884716)
4,Bethlehem Power Plant,40.62,-75.31,SKORONSKI,41.458236,-75.972972,108.504391,POINT (40.62 -75.31),POINT (41.458236 -75.972972)
...,...,...,...,...,...,...,...,...,...
60,"Vicinity Energy Philadelphia, Inc - Schuylkill...",39.94,-75.19,SKORONSKI,41.458236,-75.972972,181.116510,POINT (39.94 -75.19),POINT (41.458236 -75.972972)
61,Warren,41.84,-79.19,LOT 59,41.791883,-79.031187,14.236800,POINT (41.84 -79.19),POINT (41.791883 -79.031187)
62,Wheelabrator - Frackville,40.78,-76.18,SKORONSKI,41.458236,-75.972972,77.302950,POINT (40.78 -76.18),POINT (41.458236 -75.972972)
63,Wolf Run Energy LLC,41.61,-76.30,SKORONSKI,41.458236,-75.972972,32.076777,POINT (41.61 -76.3),POINT (41.458236 -75.972972)


In [ ]:
#@title
map

In [ ]:
#@title
pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 6.8 MB/s 
     |████████████████████████████████| 16.7 MB 8.1 MB/s 
     |████████████████████████████████| 6.3 MB 12.7 MB/s 


In [ ]:
#@title
#Library imports
import numpy as np

import geopandas as gpd
import folium
import requests
import matplotlib.pyplot as plt

from geopy.distance import geodesic
from shapely.geometry import Point

%matplotlib inline

In [ ]:
#@title
from google.colab import files # call files from google.colab
uploaded = files.upload()
uploaded_2 = files.upload()

import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['ghgp_data_2020.csv']), skiprows = 3 ,encoding='cp1252') #since windows we need the encoding to be specific
df1 = pd.read_csv(io.BytesIO(uploaded_2['Oriskany Wells Table (1).CSV'])) #since windows we need the encoding to be specific

Saving ghgp_data_2020.csv to ghgp_data_2020.csv


Saving Oriskany Wells Table (1).CSV to Oriskany Wells Table (1).CSV


In [ ]:
#@title
#What Filters do we apply
#We are only looking at PA, and we want to see all emissions by power plant
df_filt = df
df_filt = df_filt.loc[(df['State'] == 'PA')] #WE want only PA
df_filt = df_filt.drop(['Facility Id','FRS Id','Facility Name', 'City','Zip Code','Latitude', 'Longitude', 'Address', 'County','Primary NAICS Code'	], axis=1) #Drop we do not need all these
df_filt = df_filt.drop(['Is some CO2 collected on-site and used to manufacture other products and therefore not emitted from the affected manufacturing process unit(s)? (as reported under Subpart G or S)','Is some CO2 reported as emissions from the affected manufacturing process unit(s) under Subpart AA, G or P collected and transferred off-site or injected (as reported under Subpart PP)?','Does the facility employ continuous emissions monitoring? '], axis=1) #do not need
df_filt= df_filt.replace(np.nan, 0)  #change NaN to 0
df_filt

,State,Industry Type (subparts),Industry Type (sectors),Total reported direct emissions,CO2 emissions (non-biogenic),Methane (CH4) emissions,Nitrous Oxide (N2O) emissions,HFC emissions,PFC emissions,SF6 emissions,...,Pulp and Paper Manufacturing,Silicon Carbide Production,Soda Ash Manufacturing,Titanium Dioxide Production,Underground Coal Mines,Zinc Production,Municipal Landfills,Industrial Wastewater Treatment,Manufacture of Electric Transmission and Distribution Equipment,Industrial Waste Landfills
24,PA,SS,Other,"11,023.80",0,0,0,0,0,"11,023.80",...,0,0,0,0,0,0,0,0,"11,023.80",0
121,PA,"C,HH",Waste,"60,734.55",643.5,"60,090.75",0.3,0,0,0,...,0,0,0,0,0,0,"60,090.50",0,0,0
159,PA,"C,MM-REF,Y","Petroleum Product Suppliers,Refineries","115,865.74","114,521.00",792.25,552.49,0,0,0,...,0,0,0,0,0,0,0,0,0,0
166,PA,"C,N",Minerals,"39,873.10","39,845.40",12.5,15.2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
223,PA,"AA,C",Pulp and Paper,"87,841.18","86,465.90",381.75,993.53,0,0,0,...,"11,982.32",0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6422,PA,D,Power Plants,"89,215.03","88,610.50",221,383.53,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6459,PA,C,Power Plants,"20,240.12","20,219.30",9.5,11.32,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6468,PA,"C,N",Minerals,"27,774.81","27,751.20",10.5,13.11,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6493,PA,C,Waste,"160,220.04","151,974.00","3,215.50","5,030.54",0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#@title
#What are the filters we can use
uniq = df_filt['Industry Type (sectors)'].unique()

#DIsplay industry types
print('Here are our industry types: \n', uniq)
#Prompt and input
x = input('What Industry Type (sectors) do you need:  ')
#If else statement that filters based on what Industry you want
#Was not sure the type of industry(sector) we needed
if x in uniq:
  df_filt = df_filt.loc[df['Industry Type (sectors)'] == uniq[list(uniq).index(x)]]
else:
  print('No Filters where applied. Please input exact value name.')

df_filt

Here are our industry types: 
 ['Other' 'Waste' 'Petroleum Product Suppliers,Refineries' 'Minerals'
 'Pulp and Paper' 'Chemicals' 'Chemicals,Industrial Gas Suppliers'
 'Power Plants' 'Metals' 'Petroleum and Natural Gas Systems'
 'Natural Gas and Natural Gas Liquids Suppliers,Petroleum and Natural Gas Systems'
 'Other,Waste' 'Metals,Waste' 'Pulp and Paper,Waste'
 'Import and Export of Equipment Containing Fluorintaed GHGs,Other'
 'Other,Power Plants'
 'Import and Export of Equipment Containing Fluorintaed GHGs,Industrial Gas Suppliers,Other'
 'Chemicals,Refineries']
What Industry Type (sectors) do you need:  Power Plants


,State,Industry Type (subparts),Industry Type (sectors),Total reported direct emissions,CO2 emissions (non-biogenic),Methane (CH4) emissions,Nitrous Oxide (N2O) emissions,HFC emissions,PFC emissions,SF6 emissions,...,Pulp and Paper Manufacturing,Silicon Carbide Production,Soda Ash Manufacturing,Titanium Dioxide Production,Underground Coal Mines,Zinc Production,Municipal Landfills,Industrial Wastewater Treatment,Manufacture of Electric Transmission and Distribution Equipment,Industrial Waste Landfills
370,PA,C,Power Plants,"15,082.69","15,066.80",7.25,8.64,0,0,0,...,0,0,0,0,0,0,0,0,0,0
457,PA,D,Power Plants,"645,890.22","645,231.60",299.83,358.79,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537,PA,C,Power Plants,"47,873.12","47,823.80",22.5,26.82,0,0,0,...,0,0,0,0,0,0,0,0,0,0
681,PA,C,Power Plants,"19,196.58","19,176.30",9.25,11.03,0,0,0,...,0,0,0,0,0,0,0,0,0,0
709,PA,D,Power Plants,"2,446,448.30","2,443,958.50","1,134.50","1,355.30",0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6151,PA,C,Power Plants,"56,953.89","56,895.80",26.5,31.59,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6345,PA,C,Power Plants,"1,524.34","1,522.40",0.75,1.19,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6422,PA,D,Power Plants,"89,215.03","88,610.50",221,383.53,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6459,PA,C,Power Plants,"20,240.12","20,219.30",9.5,11.32,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#@title
#Used to remove commas
df_filt.replace(',','', regex=True, inplace=True)
#used to select specific cols using slice and convert to float
df_filt[df_filt.columns[3:53]].astype(float)
df_filt

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


,State,Industry Type (subparts),Industry Type (sectors),Total reported direct emissions,CO2 emissions (non-biogenic),Methane (CH4) emissions,Nitrous Oxide (N2O) emissions,HFC emissions,PFC emissions,SF6 emissions,...,Pulp and Paper Manufacturing,Silicon Carbide Production,Soda Ash Manufacturing,Titanium Dioxide Production,Underground Coal Mines,Zinc Production,Municipal Landfills,Industrial Wastewater Treatment,Manufacture of Electric Transmission and Distribution Equipment,Industrial Waste Landfills
370,PA,C,Power Plants,15082.69,15066.80,7.25,8.64,0,0,0,...,0,0,0,0,0,0,0,0,0,0
457,PA,D,Power Plants,645890.22,645231.60,299.83,358.79,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537,PA,C,Power Plants,47873.12,47823.80,22.5,26.82,0,0,0,...,0,0,0,0,0,0,0,0,0,0
681,PA,C,Power Plants,19196.58,19176.30,9.25,11.03,0,0,0,...,0,0,0,0,0,0,0,0,0,0
709,PA,D,Power Plants,2446448.30,2443958.50,1134.50,1355.30,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6151,PA,C,Power Plants,56953.89,56895.80,26.5,31.59,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6345,PA,C,Power Plants,1524.34,1522.40,0.75,1.19,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6422,PA,D,Power Plants,89215.03,88610.50,221,383.53,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6459,PA,C,Power Plants,20240.12,20219.30,9.5,11.32,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#@title
Well= df1[['API14','Well Name','Surface Hole Latitude (WGS84)','Surface Hole Longitude (WGS84)']]

Facility = df[['Facility Id','State','Industry Type (sectors)', 'Facility Name','Latitude', 'Longitude']]

In [ ]:
#@title
#Filter Facility to only show what State and Industry we want
#Filter based on state PA
Facility= Facility.loc[(Facility['State'] == 'PA')] #WE want only PA
#What are the filters we can use
uniq = Facility['Industry Type (sectors)'].unique()

#DIsplay industry types
print('Here are our industry types: \n', uniq)
#Prompt and input
x = input('What Industry Type (sectors) do you need:  ')
#If else statement that filters based on what Industry you want
#Was not sure the type of industry(sector) we needed
if x in uniq:
  Facility = Facility.loc[Facility['Industry Type (sectors)'] == uniq[list(uniq).index(x)]]
else:
  print('No Filters where applied. Please input exact value name.')

Facility

Here are our industry types: 
 ['Other' 'Waste' 'Petroleum Product Suppliers,Refineries' 'Minerals'
 'Pulp and Paper' 'Chemicals' 'Chemicals,Industrial Gas Suppliers'
 'Power Plants' 'Metals' 'Petroleum and Natural Gas Systems'
 'Natural Gas and Natural Gas Liquids Suppliers,Petroleum and Natural Gas Systems'
 'Other,Waste' 'Metals,Waste' 'Pulp and Paper,Waste'
 'Import and Export of Equipment Containing Fluorintaed GHGs,Other'
 'Other,Power Plants'
 'Import and Export of Equipment Containing Fluorintaed GHGs,Industrial Gas Suppliers,Other'
 'Chemicals,Refineries']
What Industry Type (sectors) do you need:  Power Plants


,Facility Id,State,Industry Type (sectors),Facility Name,Latitude,Longitude
370,1013070,PA,Power Plants,Alpaca Energy LLC,41.66,-76.81
457,1005709,PA,Power Plants,"Armstrong Power, LLC",40.64,-79.35
537,1005525,PA,Power Plants,BELLEFIELD BOILER PLANT,40.44,-79.95
681,1012567,PA,Power Plants,Beaver Dam Energy LLC,41.65,-76.80
709,1000127,PA,Power Plants,Bethlehem Power Plant,40.62,-75.31
...,...,...,...,...,...,...
6151,1007205,PA,Power Plants,"Vicinity Energy Philadelphia, Inc - Schuylkill...",39.94,-75.19
6345,1000880,PA,Power Plants,Warren,41.84,-79.19
6422,1006714,PA,Power Plants,Wheelabrator - Frackville,40.78,-76.18
6459,1013716,PA,Power Plants,Wolf Run Energy LLC,41.61,-76.30


In [ ]:
#@title
#only want certain values
Facility = Facility[['Facility Id','Facility Name', 'Latitude','Longitude']]



In [ ]:
#@title
Well

,API14,Well Name,Surface Hole Latitude (WGS84),Surface Hole Longitude (WGS84)
0,37131200130000,SKORONSKI,41.458236,-75.972972
1,37129285780000,POOLE-EMANUEL,40.307267,-79.283519
2,37129278100000,FLICKINGER,40.345503,-79.101131
3,37129252040000,PIPER,40.244778,-79.363642
4,37129202260000,ANDREW DONITZEN (JOHN RALPH TINKEY),40.086104,-79.278721
...,...,...,...,...
340,37005216750000,HARRY L DANDO,40.618575,-79.485506
341,37005214830000,JOHN CHAMBERS,40.611012,-79.457472
342,37005212720000,CANTERBURY COAL COMPANY,40.596054,-79.453141
343,37005212010000,NELLIE C MARTIN,40.884842,-79.346818


In [ ]:
#@title
#combine df so we can print out proper calculations

result = pd.concat([Facility.reset_index(drop=1).add_suffix(''), Well.reset_index(drop=1).add_suffix('')], axis=1)
result = result.fillna(0)
result = result.astype({'Facility Id':'int'})
result



,Facility Id,Facility Name,Latitude,Longitude,API14,Well Name,Surface Hole Latitude (WGS84),Surface Hole Longitude (WGS84)
0,1013070,Alpaca Energy LLC,41.66,-76.81,37131200130000,SKORONSKI,41.458236,-75.972972
1,1005709,"Armstrong Power, LLC",40.64,-79.35,37129285780000,POOLE-EMANUEL,40.307267,-79.283519
2,1005525,BELLEFIELD BOILER PLANT,40.44,-79.95,37129278100000,FLICKINGER,40.345503,-79.101131
3,1012567,Beaver Dam Energy LLC,41.65,-76.80,37129252040000,PIPER,40.244778,-79.363642
4,1000127,Bethlehem Power Plant,40.62,-75.31,37129202260000,ANDREW DONITZEN (JOHN RALPH TINKEY),40.086104,-79.278721
...,...,...,...,...,...,...,...,...
340,0,0,0.00,0.00,37005216750000,HARRY L DANDO,40.618575,-79.485506
341,0,0,0.00,0.00,37005214830000,JOHN CHAMBERS,40.611012,-79.457472
342,0,0,0.00,0.00,37005212720000,CANTERBURY COAL COMPANY,40.596054,-79.453141
343,0,0,0.00,0.00,37005212010000,NELLIE C MARTIN,40.884842,-79.346818


In [ ]:
#@title
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345 entries, 0 to 344
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Facility Id                     345 non-null    int64  
 1   Facility Name                   345 non-null    object 
 2   Latitude                        345 non-null    float64
 3   Longitude                       345 non-null    float64
 4   API14                           345 non-null    int64  
 5   Well Name                       345 non-null    object 
 6   Surface Hole Latitude (WGS84)   345 non-null    float64
 7   Surface Hole Longitude (WGS84)  345 non-null    float64
dtypes: float64(4), int64(2), object(2)
memory usage: 21.7+ KB


In [ ]:
#@title
subset_1 = result.loc[(result['Facility Id'] <= Facility['Facility Id'].max()) & (result['Facility Id'] >= Facility['Facility Id'].min()) , 'Facility Name':'Longitude']

subset_2 = result.loc[result['API14'] <= 37131200130000	 , 'Well Name':'Surface Hole Longitude (WGS84)']


In [ ]:
#@title
subset_1

,Facility Name,Latitude,Longitude
0,Alpaca Energy LLC,41.66,-76.81
1,"Armstrong Power, LLC",40.64,-79.35
2,BELLEFIELD BOILER PLANT,40.44,-79.95
3,Beaver Dam Energy LLC,41.65,-76.80
4,Bethlehem Power Plant,40.62,-75.31
...,...,...,...
60,"Vicinity Energy Philadelphia, Inc - Schuylkill...",39.94,-75.19
61,Warren,41.84,-79.19
62,Wheelabrator - Frackville,40.78,-76.18
63,Wolf Run Energy LLC,41.61,-76.30


In [ ]:
#@title
subset_2

,Well Name,Surface Hole Latitude (WGS84),Surface Hole Longitude (WGS84)
0,SKORONSKI,41.458236,-75.972972
1,POOLE-EMANUEL,40.307267,-79.283519
2,FLICKINGER,40.345503,-79.101131
3,PIPER,40.244778,-79.363642
4,ANDREW DONITZEN (JOHN RALPH TINKEY),40.086104,-79.278721
...,...,...,...
340,HARRY L DANDO,40.618575,-79.485506
341,JOHN CHAMBERS,40.611012,-79.457472
342,CANTERBURY COAL COMPANY,40.596054,-79.453141
343,NELLIE C MARTIN,40.884842,-79.346818


In [ ]:
#@title
num_rows_f = len(subset_1['Latitude'])
facility_rows =range(0, num_rows_f)
# for row in subset_2.iloc:
#   print(row[1], row[2])
num_rows_w = len(subset_2['Surface Hole Latitude (WGS84)'])
well_rows =range(0, num_rows_w)

#our list of items
facility_nameS=[]
facility_longtitudeS = []
facility_latitudeS = []

well_nameS = []
well_longitudeS = []
well_latitudeS = []




facility_nameS=[]
facility_longtitudeS = []
facility_latitudeS = []
well_nameS = []
well_longitudeS = []
well_latitudeS = []
distance = []
#Loop calc
for facility_index in facility_rows:
  shortest_distance = None
  nearest_well_coords = 0
  nearest_well_index = 0
  name_of_facility = subset_1.iloc[facility_index][0]
  for well_index in well_rows:
    name_of_well = subset_2.iloc[well_index][0]
    facility_latitude, facility_longtitude = subset_1.iloc[facility_index][1], subset_1.iloc[facility_index][2] #fixed
    well_latitude, well_longitude = subset_2.iloc[well_index][1], subset_2.iloc[well_index][2] #changes
    facility_coords = (facility_latitude, facility_longtitude)
    well_coords = (well_latitude, well_longitude)
    calc = (geodesic(facility_coords, well_coords).km)

    if shortest_distance == None or shortest_distance > calc: #this keeps are shortest wells
      shortest_distance = calc
      nearest_well_coords = well_coords
      nearest_well_index = well_index
      nearest_well_name = subset_2.iloc[nearest_well_index][0]

  #should append here
  facility_nameS.append(name_of_facility)
  well_nameS.append(nearest_well_name)

  facility_longtitudeS.append(facility_coords[1])
  facility_latitudeS.append(facility_coords[0])

  well_longitudeS.append(nearest_well_coords[1])
  well_latitudeS.append(nearest_well_coords[0])
  distance.append(shortest_distance)
  print(name_of_facility, nearest_well_name, facility_coords,nearest_well_coords, shortest_distance)

Alpaca Energy LLC DELCIOTTO (41.66, -76.81) (41.576682, -76.884716) 11.153909010544156
Armstrong Power, LLC T C CROWNOVER (40.64, -79.35) (40.611703, -79.437122) 8.013133029269202
BELLEFIELD BOILER PLANT FAWN DEVELOPERS INC (40.44, -79.95) (40.648018, -79.777763) 27.321533290745165
Beaver Dam Energy LLC DELCIOTTO (41.65, -76.8) (41.576682, -76.884716) 10.778206316099862
Bethlehem Power Plant SKORONSKI (40.62, -75.31) (41.458236, -75.972972) 108.50439145003006
Birdsboro Power Project SKORONSKI (40.27, -75.8) (41.458236, -75.972972) 132.75840505439314
Broad Mountain MAURICE D BIEBER B (40.67, -76.38) (41.433081, -76.838958) 93.1139552996925
Bruce Mansfield J KOEHLER (40.63, -80.42) (40.6654777, -80.3273974) 8.767483236181347
Brunner Island, LLC HESS (40.1, -76.7) (40.233403, -78.107708) 120.82687171010106
Brunot Island Power Station A SKRABA (40.46, -80.04) (40.4251849, -80.3057474) 22.875044118415577
CPV Fairview Energy Center LLC ROD & GUN (40.42, -78.86) (40.427982, -78.922226) 5.3545

In [ ]:
#@title
#Dictionary for our calculated data

data = {
    'Facility_Names': facility_nameS,
    'Facility_Latitude': facility_latitudeS,
    'Facility_Longitiude': facility_longtitudeS,
    'Nearest Well Names': well_nameS,
    'Well_Latitude': well_latitudeS,
    'Well_Longitiude': well_longitudeS,
    'Shortest Distance(km)': distance
}

df_map= pd.DataFrame.from_dict(data)

In [ ]:
#@title
#last step is to plot our dataframe on a PA map
df_map["Facility Coordinates"] = list(zip(df_map.Facility_Latitude, df_map.Facility_Longitiude ))
df_map["Well Coordinates"] = list(zip(df_map.Well_Latitude, df_map.Well_Longitiude))
df_map["Facility Coordinates"] = df_map["Facility Coordinates"].apply(Point)
df_map["Well Coordinates"] = df_map["Well Coordinates"].apply(Point)

crs={'init':'epsg:4326'} #a CRS tells Python how those coordinates are related to places on the Earth


In [ ]:
#@title
facility_geometry=[Point(xy) for xy in zip(df_map["Facility_Longitiude"], df_map["Facility_Latitude"])] #for facility cordinates

geodata=gpd.GeoDataFrame(data,crs=crs, geometry=facility_geometry)



well_geometry=[Point(xy) for xy in zip(df_map["Well_Longitiude"], df_map["Well_Latitude"])] #for well cordinates


geodata=gpd.GeoDataFrame(data,crs=crs, geometry=well_geometry)




/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
#@title
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
state_df = gpd.read_file("https://datascience.quantecon.org/assets/data/cb_2016_us_state_5m.zip")
county_df = gpd.read_file("https://datascience.quantecon.org/assets/data/cb_2016_us_county_5m.zip")
county_df = county_df.query("STATEFP == '42'") #PA FIP code is 42

In [ ]:
#@title
#Facility
facility_locations = geodata[['Facility_Latitude',	'Facility_Longitiude']]
facility_list = facility_locations.values.tolist()
len(facility_list)
facility_list[7]

#Well
well_locations = geodata[['Well_Latitude',	'Well_Longitiude']]
well_list = well_locations.values.tolist()
len(well_list)
well_list[7]

[40.6654777, -80.3273974]

In [ ]:
#@title

map = folium.Map(location=[41.5, -77.0], zoom_start=7.3)
for point in range(0, len(facility_list)):
    folium.Marker(facility_list[point],
                  icon=folium.Icon(color="blue", icon="info-sign"),
                  popup=geodata['Facility_Names'][point]).add_to(map)


for point in range(0, len(well_list)):
    folium.Marker(well_list[point],
                  icon=folium.Icon(color="red", icon="info-sign"),
                  popup=geodata['Nearest Well Names'][point]).add_to(map)
    folium.PolyLine((facility_list[point],well_list[point])).add_to(map)

In [ ]:
#@title
df_map.loc[df_map['Nearest Well Names'] == 'RODNEY WOOD']

,Facility_Names,Facility_Latitude,Facility_Longitiude,Nearest Well Names,Well_Latitude,Well_Longitiude,Shortest Distance(km),Facility Coordinates,Well Coordinates
23,Hamilton Liberty LLC,41.77,-76.48,RODNEY WOOD,41.967409,-76.595665,23.936899,POINT (41.77 -76.48),POINT (41.967409 -76.595665)
39,Milan Energy LLC,41.89,-76.59,RODNEY WOOD,41.967409,-76.595665,8.610794,POINT (41.89 -76.59),POINT (41.967409 -76.595665)
